In [1]:
pip install gurobipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: C:\Users\lifen\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from gurobipy import GRB
import gurobipy as gb

In [3]:
def find_model_optimization():
    # Create optimization model
    model = gb.Model("OptimizationModel")

    interest_rates = [1.0175, 1.0225, 1.0275]

    # Create variables
    borrow_may = model.addVars(3, lb=0, vtype=gb.GRB.CONTINUOUS, name="Borrow_May")
    borrow_jun = model.addVars(2, lb=0, vtype=gb.GRB.CONTINUOUS, name="Borrow_Jun")
    borrow_jul = model.addVars(1, lb=0, vtype=gb.GRB.CONTINUOUS, name="Borrow_Jul")

    wealth = model.addVars(4, lb=0, vtype=gb.GRB.CONTINUOUS, name="Wealth")

    # Objective function
    objective_expr = gb.quicksum(borrow_may[i] * interest_rates[i] for i in range(3))
    objective_expr += gb.quicksum(borrow_jun[i] * interest_rates[i] for i in range(2))
    objective_expr += borrow_jul[0] * interest_rates[0]

    model.setObjective(objective_expr, gb.GRB.MINIMIZE)


    # Add constraints
    for i in range(3):
        model.addConstr(wealth[0] == 140000 + 180000 + borrow_may[0] + borrow_may[1] + borrow_may[2] - 300000, f"May_Constraint_{i}")
    for i in range(2):
        model.addConstr(wealth[1] == wealth[0] + borrow_jun[0] + borrow_jun[1] + 260000 - interest_rates[0]*borrow_may[0] - 400000, f"Jun_Constraint_{i}")
    model.addConstr(wealth[2] == wealth[1] + borrow_jul[0] + 420000 - 350000 - interest_rates[1]*borrow_may[1] - borrow_jun[0]*interest_rates[0], "Jul_Constraint")
    model.addConstr(wealth[3] == wealth[2] + 580000 - 200000 - borrow_may[2]*interest_rates[2] - borrow_jun[1]*interest_rates[1] - borrow_jul[0]*interest_rates[0], "Aug_Constraint")

    # Define constraints as upper bounds in the decision variables
    model.addConstr(borrow_may[0] + borrow_may[1] + borrow_may[2] <= 250000, "Borrowing_May")
    model.addConstr(borrow_jun[0] + borrow_jun[1] <= 150000, "Borrowing_June")
    model.addConstr(borrow_jul[0] <= 350000, "Borrowing_July")

    # Cash balance constraints
    model.addConstr(wealth[0] >= 25000, "Cash_Constraint_May")
    cash_jun = model.addConstr(wealth[1] >= 20000, "Cash_Constraint_June")
    model.addConstr(wealth[2] >= 35000, "Cash_Constraint_July")
    model.addConstr(wealth[3] >= 18000, "Cash_Constraint_Aug")
    model.addConstr(wealth[2] >= (wealth[0] + wealth[1]) * 0.65, "Cash_Ratio_Constraint")

    #2b)
    b = model.addConstr(wealth[1] == wealth[0] + borrow_jun[0] + borrow_jun[1] + 260000 - interest_rates[0] * borrow_may[0] - 400000, "Cash_Constraint_June_End")


    return model, wealth,cash_jun,b

# Create the model
optimization_model, wealth, cash_jun, b = find_model_optimization()

# Optimize the model
optimization_model.optimize()

# Access wealth from outside the function
print(optimization_model.printAttr('X'))

print("Wealth values:", [wealth[i].x for i in range(4)])

print("Constr name:", b.ConstrName)
print("Constr RHS:", b.RHS)

print("Additional repayment after $7.5k increase in Jun cash: ", cash_jun.pi*7500)
print("New Total repayment: ", cash_jun.pi*7500 + optimization_model.objVal)

Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 16 rows, 10 columns and 50 nonzeros
Model fingerprint: 0x3d373705
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+04, 4e+05]
Presolve removed 9 rows and 1 columns
Presolve time: 0.01s
Presolved: 7 rows, 9 columns, 26 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.812500e+04   0.000000e+00      0s
       6    1.4290473e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.03 seconds (0.00 work units)
Optimal objective  1.429047297e+05

    Variable            X 
-----------------------

In [5]:
def find_dual_model():
    # Create optimization model
    interest_rates = [1.0175, 1.0225, 1.0275]
    dual_model = gb.Model("DualModel")

    # Create dual variables
    dual_vars = dual_model.addVars(5, lb=0, vtype=gb.GRB.CONTINUOUS, name="Dual_Variables")

    # Objective function
    objective_expr_dual = 25000 * dual_vars[0] + 20000 * dual_vars[1] + 35000 * dual_vars[2] + 18000 * dual_vars[3] + 0.65 * (dual_vars[4] - 1)

    dual_model.setObjective(objective_expr_dual, gb.GRB.MAXIMIZE)

    # Add constraints
    dual_model.addConstr(dual_vars[0] - dual_vars[1] + dual_vars[2] - dual_vars[3] == 1, "Constraint_1")
    dual_model.addConstr(dual_vars[0] - interest_rates[0] * dual_vars[1] - interest_rates[1] * dual_vars[2] - interest_rates[2] * dual_vars[3] == 0, "Constraint_2")
    dual_model.addConstr(dual_vars[4] + interest_rates[0] * dual_vars[2] + interest_rates[1] * dual_vars[3] == 0, "Constraint_3")

    return dual_model, dual_vars

# Create the dual model
dual_optimization_model, dual_vars = find_dual_model()

# Optimize the dual model
dual_optimization_model.optimize()

# Display the results
print("Dual Variables:", [dual_vars[i].x for i in range(5)])
print("Dual Objective Value:", dual_optimization_model.objVal)


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 5 columns and 11 nonzeros
Model fingerprint: 0x3b112208
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e-01, 4e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 3 rows and 5 columns
Presolve time: 0.03s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5964279e+06   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.03 seconds (0.00 work units)
Optimal objective  2.596427921e+06
Dual Variables: [58.14285714285685, 57.14285714285685, 0.0, 0.0, 0.0]
Dual Objective Value: 2596427.921428558


In [8]:
from gurobipy import GRB, Model

# Create the optimization model
model = Model("Sunnyshore Bay Dual")

# Create decision variables
y = model.addVars(3, lb=0, vtype=GRB.CONTINUOUS, name="Borrowing_dual_variables")

# Set the coefficients in the objective function
c = [1.0175, 1.0225, 1.0275]

# Set the RHS values for the constraints
RHS_y = [-250000, -150000, -350000]

# Set the coefficients for the borrowing constraints
coeff_borrowing = [[-1, -1, -1, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, -1, -1, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, -1, 0, 0, 0, 0]]

# Set the objective function
model.setObjective(y.prod(c), GRB.MAXIMIZE)

# Add constraints
for i in range(3):
    model.addConstr(sum(coeff_borrowing[i][j] * y[j] for j in range(3)) == RHS_y[i])

# Solve the model
model.optimize()

# Retrieve the optimal values of y
y_values = [y[i].getAttr('X') for i in range(3)]

print("Optimal values of y:")
print(y_values)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 3 columns and 3 nonzeros
Model fingerprint: 0xce24b7c6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+05, 4e+05]
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible or unbounded model


AttributeError: Unable to retrieve attribute 'X'

In [10]:
import numpy as np
# Create the optimization model
model = gb.Model("Sunnyshore Bay Dual")

# Create the three classes of decision variables where each Python
# variable represents a different number of Gurobi decision variables
y = model.addVars(3, lb=0, vtype=GRB.CONTINUOUS, name="Borrowing dual variables")
z = model.addVars(4, lb=0, vtype=GRB.CONTINUOUS, name="Cash balance dual variables")
w = model.addVars(4, vtype=GRB.CONTINUOUS, name="Cash equality dual variables")
v = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="Cash ratio dual variables")

for i in range(4):
    w[i].VarName = f"w[{i}]"
    w[i].lb = -1e200  # Large negative value
    w[i].ub = 1e200   # Large positive value

# The objective function
c = [1.0175,1.0225,1.0275,1.0175,1.0225,1.0175,0,0,0,0]

RHS_y = [-250000,-150000,-350000]
RHS_z = [25000,20000,35000,18000]
RHS_w = [20000,-140000,70000,380000]
RHS_v = [0]

coeff_borrowing = [[-1,-1,-1,0,0,0,0,0,0,0],
                   [0,0,0,-1,-1,0,0,0,0,0],
                   [0,0,0,0,0,-1,0,0,0,0]]

coeff_cash_bal = [[0,0,0,0,0,0,1,0,0,0],
                  [0,0,0,0,0,0,0,1,0,0],
                  [0,0,0,0,0,0,0,0,1,0],
                  [0,0,0,0,0,0,0,0,0,1]]

coeff_cash_eq =  [[-1,-1,-1,0,0,0,1,0,0,0],
                  [1.0175,0,0,-1,-1,0,-1,1,0,0],
                  [0,1.0225,0,1.0175,0,-1,0,-1,1,0],
                  [0,0,1.0275,0,1.0225,1.0175,0,0,-1,1]]

coeff_cash_ratio = [[0,0,0,0,0,0,-0.65,-0.65,1,0]]

model.setObjective(gb.quicksum(RHS_y[i]*y[i] for i in range(3)) + gb.quicksum(RHS_z[j]*z[j] for j in range(4))
                   + gb.quicksum(RHS_w[k]*w[k] for k in range(4)) + RHS_v[0]*v, GRB.MAXIMIZE)

# Add the constraints
coeff = np.transpose(np.vstack((coeff_borrowing, coeff_cash_bal, coeff_cash_eq, coeff_cash_ratio)))

for i in range(10):
    model.addConstr((gb.quicksum(y[j]*coeff[i,j] for j in range(3)) +
                        gb.quicksum(z[m]*coeff[i,m+3] for m in range(4)) +
                        gb.quicksum(w[n]*coeff[i,n+7] for n in range(4)) +
                        v*coeff[i,11] <= c[i]), "Dual Constraint")

# Optimally solve the problem
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 10 rows, 12 columns and 32 nonzeros
Model fingerprint: 0xeed37ad4
Coefficient statistics:
  Matrix range     [7e-01, 1e+00]
  Objective range  [2e+04, 4e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 4 rows and 5 columns
Presolve time: 0.00s
Presolved: 6 rows, 7 columns, 26 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.3000000e+35   9.925000e+30   2.300000e+05      0s
       5    1.4290473e+05   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.429047297e+05


In [11]:
from gurobipy import GRB, Model

# Create the optimization model
model = Model("Sunnyshore Bay Dual")

# Create decision variables
y = model.addVars(3, lb=0, vtype=GRB.CONTINUOUS, name="Borrowing_dual_variables")

# Set the coefficients in the objective function
c = [1.0175, 1.0225, 1.0275]

# Set the RHS values for the constraints
RHS_y = [-250000, -150000, -350000]

# Set the coefficients for the borrowing constraints
coeff_borrowing = [[-1, -1, -1, 0, 0, 0, 0, 0, 0, 0],
                   [0, 0, 0, -1, -1, 0, 0, 0, 0, 0],
                   [0, 0, 0, 0, 0, -1, 0, 0, 0, 0]]

# Set the objective function
model.setObjective(y.prod(c), GRB.MAXIMIZE)

# Add constraints
for i in range(3):
    model.addConstr(sum(coeff_borrowing[i][j] * y[j] for j in range(3)) == RHS_y[i])

# Solve the model
model.optimize()

# Retrieve the optimal values of y
y_values = [y[i].x for i in range(3)]

print("Optimal values of y:")
print(y_values)


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 3 columns and 3 nonzeros
Model fingerprint: 0xce24b7c6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+05, 4e+05]
Presolve time: 0.00s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible or unbounded model


AttributeError: Unable to retrieve attribute 'x'